In [1]:
import sys
sys.path.append(r'D:/projects/singletrader/')
# import singletrader
# from singletrader.datautils.qlibapi.constructor.base import MultiFactor
import pandas as pd
import warnings
from singletrader.factortesting.factor_testing import FactorEvaluation
from pprint import pprint
import numpy as np
warnings.filterwarnings('ignore')
import plotly.express as px

{"level": "INFO", "time": "2023-03-03 18:57:25,035", "timeMillis": 1677841044847,"module": "singletrader-service", "method": "python-service", "addition": "[__init__.py 45 init]", "message": "开始初始化路径..."}
[19872:MainThread](2023-03-03 18:57:25,327) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
{"level": "INFO", "time": "2023-03-03 18:57:25,327", "timeMillis": 1677841044847,"module": "qlib.Initialization", "method": "python-service", "addition": "[config.py 416 set]", "message": "default_conf: client."}
[19872:MainThread](2023-03-03 18:57:25,522) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
{"level": "INFO", "time": "2023-03-03 18:57:25,522", "timeMillis": 1677841044847,"module": "qlib.Initialization", "method": "python-service", "addition": "[__init__.py 74 init]", "message": "qlib successfully initialized based on client settings."}
[19872:MainThread](2023-03-03 18:57:25,523) INFO - qlib.Initialization -

auth success 


In [2]:
#合并数据示例
from pathlib import Path
from workflow import MultiFactorTesting
__file__ = r'D:\projects\singletrader\samples\double_sort_framework\workflow.ipynb'
file = Path(__file__)
parent_path = file.parent
data_file = parent_path.__str__() + '/data/' + r'price_and_factor_data.csv'
merged_data = pd.read_csv(data_file)
merged_data['date'] = pd.to_datetime(merged_data['date']) #date字段需要datetime格式；
merged_data = merged_data.set_index(['date','asset']) #必须为date,asset双重索引；
merged_data = merged_data.dropna()
merged_data.dropna().head()

fields = []
names = []

features = ['skew',	'distance',	'mom3M', 'turnover3M']
fields += ['$close','$open','$high','$low','$avg','$volume','$circulating_market_cap','$turnover_ratio']
bars = ['close','open','high','low','avg','volume','market_cap','turnover_ratio']
names += bars

In [3]:
fe = FactorEvaluation(merged_data[bars],merged_data[features],freq=12)

## Notes
universe: all A share

period: 2010.01~2022.12 (156 months)

$ forward\_return = \frac{close_{next\_month}}{open_{next\_month}}-1$

$ distance =  1 - close / MAX(high,252days)$

mom3M: current close / close 3 months ago

turnover3M: sum of last 3 months turnover ratio

# Part I IC analysis

In [4]:
normal_ic = fe.get_factor_ic(base='close-open',add_shift=0,method='normal')
rank_ic = fe.get_factor_ic(base='close-open',add_shift=0,method='rank')
res_ic = pd.DataFrame(columns=normal_ic.columns)

{"level": "INFO", "time": "2023-03-03 18:57:26,611", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 206 get_factor_ic]", "message": "正在计算因子skew的ic..."}
{"level": "INFO", "time": "2023-03-03 18:57:26,682", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 209 get_factor_ic]", "message": "因子skew的ic计算完毕"}
{"level": "INFO", "time": "2023-03-03 18:57:26,682", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 206 get_factor_ic]", "message": "正在计算因子distance的ic..."}
{"level": "INFO", "time": "2023-03-03 18:57:26,740", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 209 get_factor_ic]", "message": "因子distance的ic计算完毕"}
{"level": "INFO", "time": "2023-03-03 18:57:26,740", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testin

In [5]:
for factor in normal_ic.columns:
    # print('different group return of '+factor,'\n')
    ic_mean = pd.concat([normal_ic[factor],normal_ic[factor].rolling(12).mean()],axis=1)
    ic_mean.columns = ['ic','ic'+'_ma12M']
    p = px.line(ic_mean,title='normal ic of '+factor)
    p.show()


In [6]:
res_ic.loc['ic.avg'] = normal_ic.mean()
res_ic.loc['ic.std'] = normal_ic.std()
res_ic.loc['icir'] = normal_ic.mean() / normal_ic.std()
res_ic.loc['rankic'] = rank_ic.mean()

In [7]:
res_ic

,skew_1M,distance_1M,mom3M_1M,turnover3M_1M
ic.avg,-0.021928,0.002196,-0.039379,-0.038073
ic.std,0.056231,0.139505,0.120595,0.135001
icir,-0.389963,0.015743,-0.326539,-0.282020
rankic,-0.044935,0.012295,-0.061586,-0.054001


# Part II group return analysis

In [8]:
from singletrader.performance.common import performance_indicator
r = fe.get_group_returns(add_shift=0,groups=5,return_weight=True,base='close-open')

{"level": "INFO", "time": "2023-03-03 18:57:28,316", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 175 get_group_returns]", "message": "正在计算因子skew的分组收益..."}
{"level": "INFO", "time": "2023-03-03 18:57:29,490", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 181 get_group_returns]", "message": "因子skew的分组收益计算结束"}
{"level": "INFO", "time": "2023-03-03 18:57:29,491", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 175 get_group_returns]", "message": "正在计算因子distance的分组收益..."}
{"level": "INFO", "time": "2023-03-03 18:57:30,651", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "addition": "[factor_testing.py 181 get_group_returns]", "message": "因子distance的分组收益计算结束"}
{"level": "INFO", "time": "2023-03-03 18:57:30,651", "timeMillis": 1677841044847,"module": "root", "method": "python-service", "ad

In [9]:
mkt_return=merged_data.dropna().groupby(level=0).apply(lambda x:(x['close']/x['open']-1).mean()).shift(-1)

In [10]:
for factor in r[0].columns:
    print('different group return of '+factor,'\n')
    _per = performance_indicator((1+r[0][factor].unstack()).cumprod(),freq=12,language='en',mkt_return=mkt_return)
    _per.loc['turnover per trade'] = r[1][factor].groupby(level=0).apply(lambda x:x.diff().abs().sum(axis=1)/2).droplevel(0).unstack().T.mean()
    pprint(round(_per,4))
    print('\n')

different group return of skew 

                           0         1         2         3         4  \
abs_ret             3.680618   4.72928  3.007539  1.117223  0.529885   
ret_ann              0.15973  0.174094  0.145649  0.093653  0.067362   
sigma_ann           0.279565  0.277423  0.273343  0.265201  0.261148   
max_drawdown        0.480941  0.391101  0.452428  0.614188  0.597972   
SR rf=0.02          0.499812  0.555447  0.459676  0.277726  0.181362   
calmar_ratio         0.33212  0.445138  0.321927  0.152483  0.112651   
ann.return 1Y      -0.032854 -0.093856 -0.113978 -0.156634 -0.165995   
ann.return 2Y       0.130258  0.067555  0.068124  0.033436  0.021488   
ann.return 3Y       0.185664  0.132253   0.12014  0.098369  0.086514   
ann.return 5Y       0.098581  0.061161  0.042552  0.018773  0.011687   
excess_ann          0.032766   0.04713  0.018685 -0.033311 -0.059602   
excess_stdev_ann    0.042174  0.039004  0.032549  0.041877  0.043698   
IR                   0.77692  1

In [18]:
dd = r[0]['skew'].unstack()

In [19]:
dd['ss'] = dd[4]-dd[0]

In [20]:
dd

,0,1,2,3,4,hml,lmh,ss
date,,,,,,,,
2010-01-31,0.064950,0.060109,0.073205,0.065655,0.073250,0.008300,-0.008300,0.008300
2010-02-28,0.055989,0.037855,0.036237,0.024408,0.028189,-0.027800,0.027800,-0.027800
2010-03-31,-0.055903,-0.074915,-0.065449,-0.055730,-0.085021,-0.029118,0.029118,-0.029118
2010-04-30,-0.088324,-0.076217,-0.065159,-0.043278,-0.057884,0.030441,-0.030441,0.030441
2010-05-31,-0.090115,-0.078446,-0.085843,-0.072396,-0.089253,0.000862,-0.000862,0.000862
...,...,...,...,...,...,...,...,...
2022-07-31,-0.017701,-0.021189,-0.030889,-0.030187,-0.035641,-0.017939,0.017939,-0.017939
2022-08-31,-0.084551,-0.085900,-0.089652,-0.088639,-0.086342,-0.001792,0.001792,-0.001792
2022-09-30,0.020703,0.028085,0.034762,0.033773,0.038973,0.018270,-0.018270,0.018270


In [21]:
dd.corr(


)

,0,1,2,3,4,hml,lmh,ss
0,1.000000,0.982013,0.977441,0.968118,0.963442,-0.367034,0.367034,-0.367034
1,0.982013,1.000000,0.981096,0.966935,0.967506,-0.286155,0.286155,-0.286155
2,0.977441,0.981096,1.000000,0.976980,0.976149,-0.239173,0.239173,-0.239173
3,0.968118,0.966935,0.976980,1.000000,0.975751,-0.205946,0.205946,-0.205946
4,0.963442,0.967506,0.976149,0.975751,1.000000,-0.104398,0.104398,-0.104398
hml,-0.367034,-0.286155,-0.239173,-0.205946,-0.104398,1.000000,-1.000000,1.000000
lmh,0.367034,0.286155,0.239173,0.205946,0.104398,-1.000000,1.000000,-1.000000
ss,-0.367034,-0.286155,-0.239173,-0.205946,-0.104398,1.000000,-1.000000,1.000000
